In [ ]:
from experiment import run_experiment

In [ ]:
run = {
    # Dictionaries and other stuff
    'save_dict': False,
    'dict': 'runs_report.xlsx',
    'from_scratch' : True,

    # Data preprocessing
    'amplitudes': ['050'],
    'seq_size': 500,
    'train_percentage': 0.9,
    'input_downsample_factor': 3,
    'downsampling_strat': 'uniform',
    'interpolator': 'linear', #     (*)

    # Model
    'model_name' : 'mlp_linear',
    'model': 'mlp',
    'integ_strat': 'fixed-grid', # ['fixed-grid', 'adaptive']   (*)
    'integ_method': 'rk4', # ['euler', 'rk4']   (*)
    'deriv_model': 'mlp', # [ 'mlp' ]   (*)
    'hidd_sizes': [1],
    'nonlinearity': 'tanh', # ['tanh', 'relu', 'sigmoid', 'softplus', 'elu', 'selu', 'swish']
    'time-dep':'none', # ['time-att', 'time-branch']
    'time_hidd_sizes': [1,3,1],
    'initializer': 'xavier',

    # Training
    'epochs': 600,
    'batch_size': 3000, # (**)
    'learning_rate': 0.001,
}



In [ ]:
# (1) stick all of them together in validation, restore model params and .apply(params, model). 
# (2) Take difference with output and prediction and show it for a limited sequence. 
# (3) Compute ffts of output and compare with time series of acoustics, fft of acoustics and high selected frequencies acoustics (entropy + energy)
# (4) Regression ??? Sindy ??

model, params = run_experiment(run)

In [1]:
from torch_version.models.mlp import mlp
from torch_version.modules.ingestion_prepro_other import ingestion_preprocess
from torch_version.modules.update_params import update_weights

dataset, _, input_size = ingestion_preprocess(['050'], 100, 3, 1.0, 'uniform')

model = mlp(input_sizes=[input_size, 1], nonlinearity='tanh', time_dependency='none', time_sizes=[1, 3, 1])

model = update_weights(model, "mlp_comparison")



In [6]:
from modules.ingestion_prepro_other import ingestion_preprocess
import matplotlib.pyplot as plt

"""run = {
    # Data preprocessing
    'amplitudes': ['050'],
    'train_percentage': 1.0,
    'input_downsample_factor': 3,
    'downsampling_strat': 'uniform',
}"""

#dataset, _, _ = ingestion_preprocess(run)


from torch.utils.data import DataLoader
data_loader = DataLoader(dataset, batch_size=len(dataset), shuffle=False)

for inputs, outputs in data_loader:

    #preds = model.apply({'params': params}, inputs)

    preds = model(inputs)

residuals = outputs - preds

plt.plot(residuals[1000:], label='Residuals')
plt.legend()
plt.show()
plt.close()

TypeError: Cannot interpret 'torch.float32' as a data type